In [ ]:
# Starte diesen Abschnitt des Codes, damit die Nextbike-Datenbank ausgewählt wird und die entsprechenden Daten davon
# in die "city_summaries.csv" Datei gespeichert werden.

# Vor dem Start müssen Bibliotheken installiert werden.
# Folgendes muss hierzu vor dem Start in das Terminal eingegeben werden:
# pip install pandas

import sqlite3
import pandas as pd
import os

# Pfad zur Database (eine Ebene über dem Repository-Ordner)
db_path = "../nextbike_data_old.db"

# Prüfen ob die Database existiert
if not os.path.exists(db_path):
    print(f"Fehler: Database nicht gefunden unter {db_path}")
    raise SystemExit(1)

# Verbindung zur Database herstellen
conn = sqlite3.connect(db_path)

# Nur die Tabelle city_summaries wird exportiert
tables = ["city_summaries"]

for table in tables:
    try:
        # Tabelle als DataFrame laden
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

        # Timestamp parsen und date/hour extrahieren
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['date'] = df['timestamp'].dt.date
        df['hour'] = df['timestamp'].dt.hour

        # Nach date und hour gruppieren und Mittelwerte berechnen
        grouped = df.groupby(['date', 'hour']).agg({
            'total_bikes': 'mean',
            'available_bikes': 'mean',
            'booked_bikes': 'mean',
            'set_point_bikes': 'mean',
            'num_places': 'mean',
            'city_uid': 'first',
            'city_name': 'first',
            'country_name': 'first'
        }).reset_index()

        df = grouped

        # Runden der spezifischen Spalten und zu int konvertieren, um .0 zu vermeiden
        df['total_bikes'] = df['total_bikes'].round().astype(int)
        df['available_bikes'] = df['available_bikes'].round().astype(int)
        df['booked_bikes'] = df['booked_bikes'].round().astype(int)
        df['set_point_bikes'] = df['set_point_bikes'].round().astype(int)
        df['num_places'] = df['num_places'].round().astype(int)

        # Format hour as HH:MM:SS
        df['hour'] = df['hour'].apply(lambda h: f"{h:02d}:00:00")

        # Als CSV speichern
        csv_filename = f"{table}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8')

    except Exception as e:
        print(f"✗ Fehler beim Export von {table}: {e}")

# Verbindung schließen
conn.close()

In [ ]:
# Starte diesen Abschnitt des Codes, damit die entsprechenden Wetterdaten von Open-Meteo.com abgefragt werden und
# in die "weather_data.csv" Datei gespeichert werden.

# Vor dem Start müssen Bibliotheken installiert werden.
# Folgendes muss hierzu vor dem Start in das Terminal eingegeben werden:
# pip install openmeteo-requests
# pip install requests-cache retry-requests numpy pandas

import openmeteo_requests # Braucht man für den Abruf der Wetterdaten für Karlsruhe

import pandas as pd # Zur Datenanalyse
import requests_cache # Gut für API-Abfragen, damit Prozesse schneller Ablaufen
from retry_requests import retry # Bei Fehlern in der API-Abfrage

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 49.0094,
	"longitude": 8.4044,
	"start_date": "2025-09-14",
	"end_date": "2025-10-10",
	"hourly": ["temperature_2m", "rain", "snowfall", "relative_humidity_2m", "cloud_cover", "wind_speed_10m", "sunshine_duration", "is_day"],
	"timezone": "Europe/Berlin",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(6).ValuesAsNumpy()
hourly_is_day = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m (°C)"] = hourly_temperature_2m
hourly_data["rain (mm)"] = hourly_rain
hourly_data["snowfall (mm)"] = hourly_snowfall
hourly_data["relative_humidity_2m (%)"] = hourly_relative_humidity_2m
hourly_data["cloud_cover (%)"] = hourly_cloud_cover
hourly_data["wind_speed_10m (km/h)"] = hourly_wind_speed_10m
hourly_data["sunshine_duration (seconds)"] = hourly_sunshine_duration
hourly_data["is_day (1=yes / 0=no)"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Split date into date and hour
hourly_dataframe['hour'] = hourly_dataframe['date'].dt.strftime('%H:%M:%S')
hourly_dataframe['date'] = hourly_dataframe['date'].dt.date

# Reorder columns to have hour right after date
cols = ['date', 'hour'] + [col for col in hourly_dataframe.columns if col not in ['date', 'hour']]
hourly_dataframe = hourly_dataframe[cols]

# Save data to CSV
hourly_dataframe.to_csv("weather_data.csv", index=False)